In [1]:
import nflfastpy as npy
import urllib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.patheffects as pe
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
from PIL import Image

In [2]:
font_paths = [
    "../../fonts/Poppins-Regular.ttf",
    "../../fonts/Poppins-Medium.ttf",
    "../../fonts/Poppins-SemiBold.ttf",
    "../../fonts/Poppins-Bold.ttf"
]

for path in font_paths:
    fm.fontManager.addfont(path)
    fm.FontProperties(fname=path)

In [3]:
df_2021 = npy.load_pbp_data(2021)
df_roster_2021 = npy.load_roster_data(2021)
df_team = npy.load_team_logo_data()
df_schedule = npy.load_schedule_data(2021)

In [4]:
df_home = (
    df_schedule
    .groupby("home_team", as_index=False)
    .agg(net_home=("home_result", "sum"), pf_home=("home_score", "sum"))
)

df_away = (
    df_schedule
    .groupby("away_team", as_index=False)
    .agg(net_away=("home_result", "sum"), pf_away=("away_score", "sum"))
)

df_net = df_home.merge(df_away, left_on="home_team", right_on="away_team")
df_net["net_pts"] = df_net["net_home"] - df_net["net_away"]
df_net["pf"] = df_net["pf_home"] + df_net["pf_away"]
df_net = (
    df_net
    .drop(columns=["net_home", "away_team", "net_away", "pf_home", "pf_away"])
    .rename(columns={"home_team": "team"})
)
df_net

team  net_pts   pf
0   ARI       91  419
1   ATL     -136  293
2   BAL       -2  374
3   BUF      177  456
4   CAR      -76  287
5   CHI      -82  294
6   CIN       89  444
7   CLE      -27  328
8   DAL      147  479
9   DEN       17  311
10  DET     -149  288
11   GB       86  420
12  HOU     -169  255
13  IND      101  440
14  JAX     -219  227
15   KC      112  452
16   LA       91  436
17  LAC       18  442
18   LV      -68  339
19  MIA      -41  308
20  MIN      -15  394
21   NE      168  438
22   NO       19  334
23  NYG     -143  251
24  NYJ     -177  300
25  PHI       84  418
26  PIT      -58  327
27  SEA       21  357
28   SF       59  400
29   TB      134  470
30  TEN       62  391
31  WAS     -114  313

In [5]:
df_passing = df_2021[(df_2021["pass_attempt"] == 1) & (df_2021["two_point_attempt"] == 0)].copy()
df_passing = (
    df_passing.groupby(["passer_player_id", "week"], as_index=False)
    .agg(
        team=("posteam", "first"),
        opponent=("defteam", "first"),
        completions=("complete_pass", "sum"),
        dropbacks=("pass_attempt", "sum"),
        yards=("passing_yards", "sum"),
        total_yards=("yards_gained", "sum"),
        tds=("pass_touchdown", "sum"),
        ints=("interception", "sum"),
        sacks=("sack", "sum")
    )
    .merge(df_roster_2021[["full_name", "gsis_id"]], left_on="passer_player_id", right_on="gsis_id")
)

In [6]:
# Sort by Passer and Week
df_passing_cum = df_passing.sort_values(["full_name", "week"]).copy().reset_index(drop=True)

# Get Cumulative Stats
df_passing_cum["completions"] = df_passing_cum.groupby("full_name")["completions"].cumsum()
df_passing_cum["dropbacks"] = df_passing_cum.groupby("full_name")["dropbacks"].cumsum()
df_passing_cum["yards"] = df_passing_cum.groupby("full_name")["yards"].cumsum()
df_passing_cum["total_yards"] = df_passing_cum.groupby("full_name")["total_yards"].cumsum()
df_passing_cum["tds"] = df_passing_cum.groupby("full_name")["tds"].cumsum()
df_passing_cum["ints"] = df_passing_cum.groupby("full_name")["ints"].cumsum()
df_passing_cum["sacks"] = df_passing_cum.groupby("full_name")["sacks"].cumsum()

In [7]:
# Calculate ANY/A
df_passing_cum["attempts"] = df_passing_cum["dropbacks"] - df_passing_cum["sacks"]
df_passing_cum["sack_yards_lost"] = df_passing_cum["yards"] - df_passing_cum["total_yards"]
df_passing_cum["any_a"] = (
    (df_passing_cum["yards"] + (20 * df_passing_cum["tds"]) - (45 * df_passing_cum["ints"]) - df_passing_cum["sack_yards_lost"]) / 
    df_passing_cum["dropbacks"]
)
df_passing_cum["y_a"] = df_passing_cum["yards"] / df_passing_cum["attempts"]

In [8]:
df_passing_cum

passer_player_id  week team opponent  completions  dropbacks   yards  \
0         00-0035676     8  TEN      IND          0.0        1.0     0.0   
1         00-0035676    16  TEN       SF          0.0        2.0     0.0   
2         00-0023459     1   GB       NO         15.0       29.0   133.0   
3         00-0023459     2   GB      DET         37.0       59.0   388.0   
4         00-0023459     3   GB       SF         60.0       93.0   649.0   
..               ...   ...  ...      ...          ...        ...     ...   
677       00-0037013    14  NYJ       NO        160.0      313.0  1741.0   
678       00-0037013    15  NYJ      MIA        173.0      342.0  1911.0   
679       00-0037013    16  NYJ      JAX        187.0      365.0  2013.0   
680       00-0037013    17  NYJ       TB        206.0      399.0  2247.0   
681       00-0037013    18  NYJ      BUF        213.0      427.0  2334.0   

     total_yards  tds  ints  sacks      full_name     gsis_id  attempts  \
0            0.0  0.0   0.0    0.0     A.J. Brown  00-0035676       1.0   
1            0.0  0.0   0.0    0.0     A.J. Brown  00-0035676       2.0   
2          122.0  0.0   2.0    1.0  Aaron Rodgers  00-0023459      28.0   
3          349.0  4.0   2.0    4.0  Aaron Rodgers  00-0023459      55.0   
4          602.0  6.0   2.0    5.0  Aaron Rodgers  00-0023459      88.0   
..           ...  ...   ...    ...            ...         ...       ...   
677       1506.0  6.0  11.0   28.0    Zach Wilson  00-0037013     285.0   
678       1632.0  6.0  11.0   34.0    Zach Wilson  00-0037013     308.0   
679       1732.0  7.0  11.0   35.0    Zach Wilson  00-0037013     330.0   
680       1956.0  8.0  11.0   36.0    Zach Wilson  00-0037013     363.0   
681       1964.0  9.0  11.0   44.0    Zach Wilson  00-0037013     383.0   

     sack_yards_lost     any_a       y_a  
0                0.0  0.000000  0.000000  
1                0.0  0.000000  0.000000  
2               11.0  1.103448  4.750000  
3               39.0  5.745763  7.054545  
4               47.0  6.795699  7.375000  
..               ...       ...       ...  
677            235.0  3.613419  6.108772  
678            279.0  3.675439  6.204545  
679            281.0  3.772603  6.100000  
680            291.0  4.062657  6.190083  
681            370.0  3.861827  6.093995  

[682 rows x 17 columns]

In [9]:
df_passing_cum[df_passing_cum["full_name"] == "Aaron Rodgers"]

passer_player_id  week team opponent  completions  dropbacks   yards  \
2        00-0023459     1   GB       NO         15.0       29.0   133.0   
3        00-0023459     2   GB      DET         37.0       59.0   388.0   
4        00-0023459     3   GB       SF         60.0       93.0   649.0   
5        00-0023459     4   GB      PIT         80.0      132.0   897.0   
6        00-0023459     5   GB      CIN        107.0      173.0  1241.0   
7        00-0023459     6   GB      CHI        124.0      199.0  1436.0   
8        00-0023459     7   GB      WAS        151.0      237.0  1710.0   
9        00-0023459     8   GB      ARI        173.0      275.0  1894.0   
10       00-0023459    10   GB      SEA        196.0      313.0  2186.0   
11       00-0023459    11   GB      MIN        219.0      348.0  2571.0   
12       00-0023459    12   GB       LA        247.0      394.0  2878.0   
13       00-0023459    14   GB      CHI        276.0      434.0  3219.0   
14       00-0023459    15   GB      BAL        299.0      468.0  3487.0   
15       00-0023459    16   GB      CLE        323.0      502.0  3689.0   
16       00-0023459    17   GB      MIN        352.0      540.0  3977.0   
17       00-0023459    18   GB      DET        366.0      561.0  4115.0   

    total_yards   tds  ints  sacks      full_name     gsis_id  attempts  \
2         122.0   0.0   2.0    1.0  Aaron Rodgers  00-0023459      28.0   
3         349.0   4.0   2.0    4.0  Aaron Rodgers  00-0023459      55.0   
4         602.0   6.0   2.0    5.0  Aaron Rodgers  00-0023459      88.0   
5         838.0   8.0   2.0    8.0  Aaron Rodgers  00-0023459     124.0   
6        1171.0  10.0   3.0   10.0  Aaron Rodgers  00-0023459     163.0   
7        1340.0  12.0   3.0   13.0  Aaron Rodgers  00-0023459     186.0   
8        1587.0  15.0   3.0   16.0  Aaron Rodgers  00-0023459     221.0   
9        1771.0  17.0   3.0   17.0  Aaron Rodgers  00-0023459     258.0   
10       2058.0  17.0   4.0   18.0  Aaron Rodgers  00-0023459     295.0   
11       2430.0  21.0   4.0   20.0  Aaron Rodgers  00-0023459     328.0   
12       2737.0  23.0   4.0   21.0  Aaron Rodgers  00-0023459     373.0   
13       3057.0  27.0   4.0   24.0  Aaron Rodgers  00-0023459     410.0   
14       3307.0  30.0   4.0   27.0  Aaron Rodgers  00-0023459     441.0   
15       3509.0  33.0   4.0   27.0  Aaron Rodgers  00-0023459     475.0   
16       3797.0  35.0   4.0   27.0  Aaron Rodgers  00-0023459     513.0   
17       3927.0  37.0   4.0   30.0  Aaron Rodgers  00-0023459     531.0   

    sack_yards_lost     any_a       y_a  
2              11.0  1.103448  4.750000  
3              39.0  5.745763  7.054545  
4              47.0  6.795699  7.375000  
5              59.0  6.878788  7.233871  
6              70.0  7.144509  7.613497  
7              96.0  7.261307  7.720430  
8             123.0  7.392405  7.737557  
9             123.0  7.185455  7.341085  
10            128.0  7.086262  7.410169  
11            141.0  7.672414  7.838415  
12            141.0  7.657360  7.715818  
13            162.0  7.873272  7.851220  
14            180.0  7.963675  7.907029  
15            180.0  7.946215  7.766316  
16            180.0  7.994444  7.752437  
17            188.0  7.998217  7.749529

In [10]:
# Get List of Passers with > 1000 total yards
qb_list = (
    df_passing.groupby(["full_name", "team"], as_index=False)
    .agg(
        {
            "completions": "sum",
            "yards": "sum",
            "dropbacks": "sum",
            "week": "max"
        }
    )
)

rookies = [
    "Mac Jones",
    "Davis Mills",
    "Trevor Lawrence",
    "Justin Fields",
    "Zach Wilson"
]

qb_list = (
    qb_list[qb_list["yards"] >= 1000]
    .merge(df_passing_cum[["full_name", "week", "any_a"]])
    .reset_index(drop=True)
)

# Get Team Colors
qb_list = (
    qb_list.merge(df_team[["team_abbr", "team_color", "team_color2"]].rename(columns={"team_abbr": "team"}))
)

qb_list.loc[qb_list["full_name"].isin(rookies), "rookie"] = True
qb_list["rookie"] = qb_list["rookie"].fillna(False)
qb_list = qb_list.sort_values("any_a", ascending=False).reset_index(drop=True)
qb_list

full_name team  completions   yards  dropbacks  week     any_a  \
0        Aaron Rodgers   GB        366.0  4115.0      561.0    18  7.998217   
1           Joe Burrow  CIN        366.0  4611.0      571.0    17  7.514886   
2     Matthew Stafford   LA        404.0  4886.0      631.0    18  7.445325   
3         Kirk Cousins  MIN        372.0  4221.0      589.0    18  7.417657   
4            Tom Brady   TB        485.0  5316.0      741.0    18  7.411606   
5      Jimmy Garoppolo   SF        301.0  3810.0      471.0    18  7.365180   
6         Dak Prescott  DAL        410.0  4449.0      626.0    18  7.340256   
7       Jameis Winston   NO         95.0  1170.0      172.0     8  7.244186   
8       Russell Wilson  SEA        259.0  3113.0      433.0    18  7.106236   
9      Patrick Mahomes   KC        436.0  4839.0      686.0    18  7.067055   
10      Justin Herbert  LAC        443.0  5014.0      703.0    18  6.948791   
11        Kyler Murray  ARI        334.0  3818.0      513.0    18  6.943470   
12          Derek Carr   LV        428.0  4804.0      666.0    18  6.596096   
13        Carson Wentz  IND        322.0  3563.0      549.0    18  6.486339   
14          Josh Allen  BUF        409.0  4407.0      672.0    18  6.380952   
15         Jalen Hurts  PHI        265.0  3144.0      458.0    17  6.351528   
16   Teddy Bridgewater  DEN        285.0  3052.0      457.0    15  6.315098   
17           Mac Jones   NE        352.0  3801.0      549.0    18  6.220401   
18      Trevor Siemian   NO        108.0  1154.0      197.0    18  5.964467   
19           Matt Ryan  ATL        375.0  3968.0      601.0    18  5.913478   
20      Tua Tagovailoa  MIA        263.0  2653.0      409.0    18  5.797066   
21       Lamar Jackson  BAL        246.0  2882.0      420.0    14  5.778571   
22          Jared Goff  DET        332.0  3245.0      529.0    18  5.642722   
23        Daniel Jones  NYG        232.0  2428.0      383.0    12  5.621410   
24      Ryan Tannehill  TEN        357.0  3734.0      578.0    18  5.531142   
25         Davis Mills  HOU        263.0  2664.0      425.0    18  5.477647   
26  Ben Roethlisberger  PIT        390.0  3740.0      643.0    18  5.429238   
27      Baker Mayfield  CLE        253.0  3010.0      461.0    17  5.414317   
28     Taylor Heinicke  WAS        321.0  3419.0      532.0    18  5.387218   
29     Trevor Lawrence  JAX        359.0  3641.0      635.0    18  4.532283   
30         Andy Dalton  CHI        149.0  1515.0      254.0    18  4.437008   
31     Jacoby Brissett  MIA        141.0  1283.0      244.0    16  4.389344   
32         Sam Darnold  CAR        243.0  2527.0      441.0    18  4.349206   
33       Justin Fields  CHI        159.0  1870.0      306.0    15  4.235294   
34       Tyler Huntley  BAL        122.0  1081.0      206.0    18  4.145631   
35         Zach Wilson  NYJ        213.0  2334.0      427.0    18  3.861827   

   team_color team_color2  rookie  
0     #203731     #FFB612   False  
1     #FB4F14     #000000   False  
2     #003594     #FFD100   False  
3     #4F2683     #FFC62F   False  
4     #A71930     #322F2B   False  
5     #AA0000     #B3995D   False  
6     #002244     #B0B7BC   False  
7     #D3BC8D     #000000   False  
8     #002244     #69be28   False  
9     #E31837     #FFB612   False  
10    #007BC7     #ffc20e   False  
11    #97233F     #000000   False  
12    #000000     #A5ACAF   False  
13    #002C5F     #a5acaf   False  
14    #00338D     #C60C30   False  
15    #004C54     #A5ACAF   False  
16    #002244     #FB4F14   False  
17    #002244     #C60C30    True  
18    #D3BC8D     #000000   False  
19    #A71930     #000000   False  
20    #008E97     #F58220   False  
21    #241773     #9E7C0C   False  
22    #0076B6     #B0B7BC   False  
23    #0B2265     #A71930   False  
24    #002244     #4B92DB   False  
25    #03202F     #A71930    True  
26    #000000     #FFB612   False  
27    #FF3C00     #311D00   False  
28    #5A1414     #FFB612   False  
29    #00

In [11]:
league_avg = df_passing[["completions", "dropbacks", "yards", "total_yards", "tds", "ints", "sacks"]].sum()
league_avg["attempts"] = league_avg["dropbacks"] - league_avg["sacks"]
league_avg["sack_yards_lost"] = league_avg["yards"] - league_avg["total_yards"]

# Calculate ANY/A
league_avg["any_a"] = (
    (league_avg["yards"] + (20 * league_avg["tds"]) - (45 * league_avg["ints"]) - league_avg["sack_yards_lost"]) / 
    league_avg["dropbacks"]
)

In [12]:
league_avg

completions         12122.000000
dropbacks           19962.000000
yards              132831.000000
total_yards        124234.000000
tds                   840.000000
ints                  440.000000
sacks                1244.000000
attempts            18718.000000
sack_yards_lost      8597.000000
any_a                   6.073239
dtype: float64

### Get Overall Team ANY/A

In [13]:
df_team_stats = (
    df_passing.groupby("team", as_index=False)
    .sum()
)

df_team_stats["attempts"] = df_team_stats["dropbacks"] - df_team_stats["sacks"]
df_team_stats["sack_yards_lost"] = df_team_stats["yards"] - df_team_stats["total_yards"]

# Calculate ANY/A
df_team_stats["any_a"] = (
    (df_team_stats["yards"] + (20 * df_team_stats["tds"]) - (45 * df_team_stats["ints"]) - df_team_stats["sack_yards_lost"]) / 
    df_team_stats["dropbacks"]
)

# Merge with point differential
df_team_stats = df_team_stats.merge(df_net)

df_team_stats = df_team_stats.sort_values("any_a", ascending=False).reset_index(drop=True)
df_team_stats

team  week  completions  dropbacks   yards  total_yards   tds  ints  sacks  \
0    GB   194        402.0      626.0  4526.0       4315.0  39.0   7.0   33.0   
1   DAL   248        444.0      680.0  4963.0       4800.0  40.0  11.0   33.0   
2    SF   203        343.0      548.0  4437.0       4221.0  26.0  14.0   33.0   
3   CIN   213        384.0      610.0  4806.0       4403.0  36.0  14.0   55.0   
4    TB   227        492.0      754.0  5383.0       5229.0  43.0  12.0   23.0   
5    LA   196        406.0      638.0  4893.0       4642.0  41.0  18.0   31.0   
6   MIN   215        398.0      634.0  4450.0       4238.0  34.0   7.0   30.0   
7    KC   176        448.0      703.0  4937.0       4791.0  37.0  13.0   28.0   
8   LAC   183        443.0      705.0  5014.0       4800.0  38.0  15.0   31.0   
9   ARI   210        416.0      631.0  4650.0       4307.0  27.0  11.0   39.0   
10  SEA   167        324.0      541.0  3815.0       3432.0  30.0   7.0   46.0   
11   NE   202        364.0      563.0  4098.0       3857.0  24.0  13.0   28.0   
12   LV   198        429.0      668.0  4808.0       4567.0  23.0  14.0   40.0   
13  IND   159        324.0      554.0  3588.0       3361.0  27.0   8.0   32.0   
14  PHI   185        307.0      525.0  3585.0       3404.0  20.0  10.0   31.0   
15  BUF   225        415.0      682.0  4450.0       4284.0  36.0  16.0   27.0   
16  DEN   243        354.0      581.0  3856.0       3593.0  20.0   9.0   40.0   
17   NO   227        293.0      541.0  3437.0       3186.0  29.0  13.0   37.0   
18  ATL   204        377.0      614.0  3987.0       3713.0  20.0  15.0   40.0   
19  TEN   190        359.0      582.0  3745.0       3418.0  22.0  14.0   47.0   
20  CLE   186        320.0      569.0  3619.0       3320.0  21.0  14.0   49.0   
21  DET   232        396.0      629.0  3884.0       3598.0  23.0  14.0   36.0   
22  WAS   193        356.0      593.0  3746.0       3441.0  21.0  15.0   43.0   
23  BAL   202        396.0      668.0  4267.0       3961.0  21.0  18.0   57.0   
24  PIT   188        425.0      702.0  4017.0       3778.0  23.0  11.0   38.0   
25  MIA   229        404.0      656.0  3936.0       3651.0  21.0  14.0   40.0   
26  HOU   193        354.0      590.0  3630.0       3305.0  21.0  15.0   44.0   
27  NYJ   232        357.0      656.0  3959.0       3541.0  20.0  20.0   53.0   
28  JAX   173        361.0      637.0  3674.0       3436.0  12.0  17.0   32.0   
29  CHI   197        332.0      600.0  3635.0       3207.0  16.0  20.0   58.0   
30  NYG   251        351.0      631.0  3463.0       3196.0  15.0  20.0   38.0   
31  CAR   243        348.0      651.0  3573.0       3239.0  14.0  21.0   52.0   

    attempts  sack_yards_lost     any_a  net_pts   pf  
0      593.0            211.0  7.635783       86  420  
1      647.0            163.0  7.507353      147  479  
2      515.0            216.0  7.501825       59  400  
3      555.0            403.0  7.365574       89  444  
4      731.0            154.0  7.359416      134  470  
5      607.0            251.0  7.291536       91  436  
6      604.0            212.0  7.260252      -15  394  
7      675.0            146.0  7.035562      112  452  
8      674.0            214.0  6.929078       18  442  
9      592.0            343.0  6.896989       91  419  
10     495.0            383.0  6.870610       21  357  
11     535.0            241.0  6.664298      168  438  
12     628.0            241.0  6.582335      -68  339  
13     522.0            227.0  6.391697      101  440  
14     494.0            181.0  6.388571       84  418  
15     655.0            166.0  6.281525      177  456  
16     541.0            263.0  6.175559       17  311  
17     504.0            251.0  5.879852       19  334  
18     574.0            274.0  5.599349     -136  293  
19     535.0            327.0  5.546392       62  391  
20     520.0            299.0  5.465729      -27  328  
21     593.0            286.0  5.449921     -149  288  
22     550.0            305.0  5.372681  

In [14]:
df_net["net_pts"].sum()

0

In [15]:
# Get all the logos
for i, team in enumerate(df_team_stats.team):
    im = Image.open(urllib.request.urlopen(df_team[df_team["team_abbr"] == team].iloc[0].team_logo_espn))
    im.save(f"./logo-{team}.png")

In [26]:
plt.rcParams["font.family"] = "Poppins"
plt.rcParams["font.size"] = 16
plt.rcParams["xtick.major.size"] = 0
plt.rcParams["ytick.major.size"] = 0

fig = plt.figure(figsize=(20, 15), facecolor="#fbfbf0")
gs = fig.add_gridspec(nrows=3, ncols=2, hspace=0.02, wspace=0.03, height_ratios=[0.1, 0.8, 1])
ax_title = fig.add_subplot(gs[0, :], frame_on=False)
ax_formula = fig.add_subplot(gs[1, 0], frame_on=False)
ax_team_stats = fig.add_subplot(gs[2, 0], frame_on=False)
ax_league = fig.add_subplot(gs[1:3, 1], frame_on=False)

# Formula
ax_formula.text(
    0.01, 0.7,
    "Adjusted Net Yards per Attempt (ANY/A) takes the basic yards per \n" +
    "attempt calculation (Y/A) and adjusts for passing touchdowns, in-\n" +
    "-terceptions, and sacks. To do this, sack yards are subtracted from\n" +
    "passing yards, passing touchdowns are given as +20 yards, and \n" +
    "interceptions are -45 yards. Also, instead of passing attempts,\n" +
    "total dropbacks (attempts + sacks) are used.",
    ha="left",
    va="top",
    linespacing=1.4
)

ax_formula.text(
    0.5, 0.25,
    "$\mathregular{\\frac{Passing\;Yards - Sack\;Yards + (20 \\times Passing\;TDs) - (45 \\times INTs)}{Passing\;Attempts + Sacks}}$",
    ha="center",
    va="top",
    fontsize=25
)

ax_formula.set_xticks([])
ax_formula.set_yticks([])

# Performances
ax_team_stats.grid()
ax_team_stats.set_axisbelow(True)
ax_team_stats.plot([-2.5, 2.5], [0, 0], color="gray", lw=2, zorder=1)
ax_team_stats.plot([0, 0], [-15, 15], color="gray", lw=2, zorder=1)
ax_team_stats.set_xlabel("Relative ANY/A to League Average", labelpad=5)
ax_team_stats.set_ylabel("Net Points per Game", labelpad=5)
ax_team_stats.text(
    0.75, 0.95,
    "Above Average Passing Offense",
    ha="center",
    va="top",
    size=14, 
    transform=ax_team_stats.transAxes,
    path_effects=[pe.withStroke(linewidth=4, foreground="white")]
)
ax_team_stats.text(
    0.25, 0.95,
    "Below Average Passing Offense",
    ha="center",
    va="top",
    size=14,
    transform=ax_team_stats.transAxes,
    path_effects=[pe.withStroke(linewidth=4, foreground="white")]
)

# Add logos as scatter points
playoff_teams = ["GB", "TB", "DAL", "LA", "ARI", "SF", "PHI", "TEN", "KC", "BUF", "CIN", "LV", "NE", "PIT"]
for row in df_team_stats.itertuples(): 
    # Logos
    im = np.array(Image.open(f"./logo-{row.team}.png"))
    xy = (row.any_a - league_avg.any_a, row.net_pts / 17)
    
    if row.team not in playoff_teams:
        imbox = OffsetImage(im, zoom=0.05, alpha=0.25)
    else:
        imbox = OffsetImage(im, zoom=0.05)
    imbox.image.axes = ax_team_stats
    
    ab = AnnotationBbox(
        imbox, xy,
        frameon=False
    )
    
    ax_team_stats.add_artist(ab)

# League Average
ax_league.plot([3, 18], [league_avg.any_a, league_avg.any_a], linestyle="--", linewidth=3, color="gray", zorder=1)
ax_league.text(
    3, league_avg.any_a + 0.1, 
    f"League Average ({league_avg.any_a.round(2)})", 
    ha="left", 
    va="baseline",
    path_effects=[pe.withStroke(linewidth=4, foreground="white")]
)

# Dataframe to merge on all weeks and ffill
df_weeks = pd.DataFrame({"week": range(1, 19)})

# Plot Top 5 QBs
top_5_qbs = qb_list[:5]["full_name"]
midpoint_top_5 = (
    df_passing_cum[df_passing_cum["full_name"].isin(top_5_qbs)]
    .groupby("full_name", as_index=False)
    .apply(lambda grp: grp[grp["week"] == grp["week"].max()])["any_a"]
    .mean()
)
for i, qb in enumerate(top_5_qbs[::-1]):
    df_temp = (
        df_passing_cum[(df_passing_cum["full_name"] == qb) & (df_passing_cum["yards"] >= 1000)]
        .reset_index(drop=True)
        .merge(df_weeks, how="right")
        .ffill()
        .copy()
    )
    ax_league.plot(
        df_temp["week"], df_temp["any_a"], 
        color=qb_list[qb_list["full_name"] == qb].team_color.iloc[0], 
        linewidth=3, 
        markevery=[-1], 
        marker="o", 
        markersize=10, 
        markeredgewidth=2, 
        path_effects=[pe.withStroke(linewidth=6, foreground="white")],
        zorder=2
    )
    coords = (df_temp.iloc[-1].week, df_temp.iloc[-1].any_a)
    offset = (i - 2) * 0.3
    ax_league.annotate(
        f"{qb} ({df_temp.iloc[-1].any_a.round(2)})", 
        xy=coords, 
        xytext=(18.8, midpoint_top_5 + offset),
        color=qb_list[qb_list["full_name"] == qb].team_color.iloc[0],
        ha="left",
        va="center",
        arrowprops=dict(arrowstyle="-", relpos=(0, 0.5), linewidth=2, color=qb_list[qb_list["full_name"] == qb].team_color.iloc[0]),
        zorder=1
    )
    
ax_league.text(
    18.8, midpoint_top_5 + 0.9,
    "Top 5 ANY/A",
    ha="left",
    va="center",
    fontweight=600
)
    
# Plot rookie QBs
rookie_qbs = qb_list[qb_list["rookie"] == True]["full_name"]
midpoint_rookie = (
    df_passing_cum[df_passing_cum["full_name"].isin(rookie_qbs)]
    .groupby("full_name", as_index=False)
    .apply(lambda grp: grp[grp["week"] == grp["week"].max()])["any_a"]
    .mean()
)
for i, qb in enumerate(rookie_qbs[::-1]):
    df_temp = (
        df_passing_cum[(df_passing_cum["full_name"] == qb) & (df_passing_cum["yards"] >= 1000)]
        .reset_index(drop=True)
        .merge(df_weeks, how="right")
        .ffill()
        .copy()
    )
    ax_league.plot(
        df_temp["week"], df_temp["any_a"], 
        color=qb_list[qb_list["full_name"] == qb].team_color.iloc[0], 
        linewidth=3, 
        markevery=[-1], 
        marker="o", 
        markersize=10, 
        markeredgewidth=2, 
        path_effects=[pe.withStroke(linewidth=6, foreground="white")],
        zorder=2
    )
    coords = (df_temp.iloc[-1].week, df_temp.iloc[-1].any_a)
    offset = (i - 2) * 0.4
    ax_league.annotate(
        f"{qb} ({df_temp.iloc[-1].any_a.round(2)})", 
        xy=coords, 
        xytext=(18.8, midpoint_rookie + offset),
        color=qb_list[qb_list["full_name"] == qb].team_color.iloc[0],
        ha="left",
        va="center",
        arrowprops=dict(arrowstyle="-", relpos=(0, 0.5), linewidth=2, color=qb_list[qb_list["full_name"] == qb].team_color.iloc[0]),
        zorder=1
    )
    
ax_league.text(
    18.8, midpoint_rookie + 1.1,
    "Rookie ANY/A",
    ha="left",
    va="center",
    fontweight=600
)
    
# Plot rest of league in gray
for qb in qb_list["full_name"][5:]:
    df_temp = (
        df_passing_cum[(df_passing_cum["full_name"] == qb) & (df_passing_cum["yards"] >= 1000)]
        .reset_index(drop=True)
        .merge(df_weeks, how="right")
        .ffill()
        .copy()
    )
    ax_league.plot(df_temp["week"], df_temp["any_a"], linewidth=2, color="gray", alpha=0.2, zorder=1)

ax_league.set_xticklabels([])
ax_league.set_xticks(range(3, 19))
ax_league.set_xlim(2.5, 18.5)
ax_league.annotate(
    "Week", 
    xy=(9, 2.9), xytext=(12, 2.9), 
    ha="center", va="center", 
    fontsize=18, 
    color="#4e4e4e",
    arrowprops=dict(arrowstyle="-", relpos=(0, 0.5), linewidth=2, color="#4e4e4e"), 
    annotation_clip=False
)
ax_league.annotate(
    "", 
    xy=(15, 2.9), xytext=(12.8, 2.9), 
    arrowprops=dict(arrowstyle="->", linewidth=2, color="#4e4e4e"), 
    annotation_clip=False
)

ax_league.set_yticklabels([])
ax_league.set_ylim(3, 10)

# Add Title
ax_title.text(0.55, 0.5, "Ranking Passers During the 2021-22 NFL Season by ANY/A", ha="center", va="center", size=30, fontweight=600)
ax_title.set(xticks=[], yticks=[])

# Add Source
ax_league.text(
    1.4, -0.1,
    "@naveen_v92\nSource: github.com/fantasydatapros/nflfastpy",
    ha="right",
    va="baseline",
    size=14,
    transform=ax_league.transAxes
)

plt.savefig("any_a.png", dpi=300, bbox_inches="tight")
plt.show()